<a href="https://colab.research.google.com/github/mancudEOM/Ensemble-deep-learning/blob/main/testdatacafe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

drive  model_5fold_NASNetMobile_16e3.tflite  sample_data


In [ ]:
  import sys
  root_path = '/content/drive/My Drive/Mulberry Leaf Dataset/' # This is the path to where in google drive the code is stored!
  sys.path.append(root_path)

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
from keras.models import Model
from keras.layers import Input, Dense
import tensorflow as tf

In [ ]:
datasetFolderName=root_path+'dataset'
MODEL_FILENAME=root_path+"model_cv.h5"
sourceFiles=[]
classLabels=['01 ChiangMai60','02 RedKing','03 WhiteKing',
             '04 BlackOodTurkey','05 TaiwanStraberry','06 BlackAustralia',
             '07 Buriram60','08 Kamphaengsaeng42','09 TaiwanMeacho','10 ChiangMaiBuriram60']
X=[]
Y=[]

img_rows, img_cols =  224, 224
test_path=datasetFolderName+'/test/'+'/10 ChiangMaiBuriram60/'
#+'Lot_2 (250).JPG'

In [ ]:
class WeightedAverageLayer(tf.keras.layers.Layer):
    def __init__(self, w1, w2, w3, **kwargs):
        super(WeightedAverageLayer, self).__init__(**kwargs)
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3

    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1] + self.w3 * inputs[2]

In [ ]:
np.set_printoptions(suppress=True)
# Load the model
model_1 =load_model(root_path+"model_5fold_DenseNet121_16e3.h5",
                    compile=False)
model_1 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='model_5fold_DenseNet121_16e3')
model_2 = load_model(root_path+"model_5fold_NASNetMobile_16e3.h5",
                     compile=False)
model_2 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='model_5fold_NASNetMobile_16e3')
model_3 = load_model(root_path+"model_5fold_MobileNetV2_16e3_5.h5",
                     compile=False)
model_3 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='model_5fold_MobileNetV2_16e3_5')
models = [model_1, model_2, model_3]
model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
#ensemble_output = Average()(model_outputs)
ensemble_output = WeightedAverageLayer(0, 0, 1)(model_outputs)
ensemble_model= Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

In [ ]:
from keras.models import load_model
import cv2
import numpy as np


class_names = ['01 ChiangMai60','02 RedKing','03 WhiteKing',
             '04 BlackOodTurkey','05 TaiwanStraberry','06 BlackAustralia',
             '07 Buriram60','08 Kamphaengsaeng42','09 TaiwanMeacho','10 ChiangMaiBuriram60'] # fill the rest

#model = load_model(root_path+"model_5fold_DNMv2_10575325.h5")

ensemble_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

img = cv2.imread(test_path+"DSC_0003.JPG")
print(img)
img = cv2.resize(img,(224,224))
img = np.reshape(img,[1,224,224,3])

classes = np.argmax(ensemble_model.predict(img), axis = -1)

print(classes)

names = [class_names[i] for i in classes]

print(names)

[[[238 213 209]
  [255 238 235]
  [255 251 249]
  ...
  [ 97  99  93]
  [ 99 103  97]
  [ 93 100  93]]

 [[237 212 208]
  [255 238 235]
  [255 251 248]
  ...
  [114 110 105]
  [119 116 111]
  [116 116 110]]

 [[237 212 208]
  [255 238 235]
  [255 251 248]
  ...
  [144 127 124]
  [150 133 130]
  [147 132 129]]

 ...

 [[ 41  62  37]
  [ 39  63  39]
  [ 35  66  45]
  ...
  [ 69  84  63]
  [ 72  84  64]
  [ 72  84  64]]

 [[ 36  58  33]
  [ 35  61  37]
  [ 35  68  47]
  ...
  [ 67  84  63]
  [ 69  84  63]
  [ 70  85  64]]

 [[ 31  56  30]
  [ 32  61  36]
  [ 34  70  48]
  ...
  [ 66  83  62]
  [ 68  83  62]
  [ 69  84  63]]]


1/1 [==============================] - 6s 6s/step
[2]
['03 WhiteKing']


In [ ]:
from tensorflow.keras.models import save_model

In [ ]:
MODEL_FILENAMEs=root_path+"model_5fold_DNMv2_10775125.h5"
ensemble_model.save(MODEL_FILENAMEs)

In [ ]:
#import tensorflow as tf

#model = tf.keras.models.load_model(root_path+"model_5fold_DNMv2_10775125.h5")
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
#tflite_model = converter.convert()
#open("model_5fold_DNMv2_10775125.tflite", "wb").write(tflite_model)